In [1]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm

In [15]:
import rotation_matrix as rm

In [2]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

In [3]:
rtde_c, rtde_r, rtde_io_set = connect_robot()

In [1]:
import numpy as np

data = np.loadtxt('../pouring_simulation/output/Assembly1_TCP.txt', delimiter=',', skiprows=1)

In [2]:
data.shape[0]

981

In [3]:
data[500]

# convert from inches to meters
data[:, 0] = data[:, 0] * 0.0254
data[:, 1] = data[:, 1] * 0.0254

In [4]:
data[500]

array([0.01993362, 0.07887081, 0.496261  ])

In [17]:
start_pos = rtde_r.getActualTCPPose()

start_pos

In [18]:
# create list of positions
positions = []
for i in range(data.shape[0]):
    positions.append([data[i,1], -data[i,0], 0.0, 0.0, 0.0, -data[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

In [19]:
positions_converted = []
for i in range(data.shape[0]):
    positions_converted.append(rm.PoseTrans(start_pos, positions[i])) # transform from base to tool coordinates(positions[i]))

In [25]:
velocity = 0.1
acceleration = 0.1
blend_1 = 0.0
blend_i = 0.02
blend_3 = 0.0
path_pose1 = [start_pos[0], start_pos[1], start_pos[2], start_pos[3], start_pos[4], start_pos[5], velocity, acceleration, blend_1]
path = []
path.append(path_pose1)
for i in range(data.shape[0]-1):
    path.append([positions_converted[i][0], positions_converted[i][1], positions_converted[i][2], positions_converted[i][3], positions_converted[i][4], positions_converted[i][5], velocity, acceleration, blend_i])

path_pose3 = [positions_converted[-1][0], positions_converted[-1][1], positions_converted[-1][2], positions_converted[-1][3], positions_converted[-1][4], positions_converted[-1][5], velocity, acceleration, blend_3]
path.append(path_pose3)

In [26]:
# Send a linear path with blending in between - (currently uses separate script)
rtde_c.moveL(path)
rtde_c.stopScript()

[[-0.31755315085470437,
  0.3776415807801057,
  0.3636001383133041,
  -1.2357319406800302,
  -1.196658279424475,
  -1.204508997154227,
  0.1,
  0.1,
  0.0],
 [-0.3175501437831499,
  0.37764339763607113,
  0.3637916884211387,
  -1.2351961044062392,
  -1.1975213237291664,
  -1.2053550738578218,
  0.1,
  0.1,
  0.02],
 [-0.317547366782388,
  0.3776452182076999,
  0.36398324172839286,
  -1.2346598153143848,
  -1.198384287951152,
  -1.2062010640841405,
  0.1,
  0.1,
  0.02],
 [-0.31754480772966637,
  0.3776470423063676,
  0.36417479880178,
  -1.234123082597414,
  -1.199247157036103,
  -1.2070469530723826,
  0.1,
  0.1,
  0.02],
 [-0.31754245489260263,
  0.3776488695172761,
  0.36436633633627613,
  -1.2335858968531532,
  -1.2001099458376066,
  -1.2078927553812555,
  0.1,
  0.1,
  0.02],
 [-0.31754033175031854,
  0.3776507006653073,
  0.36455790043411734,
  -1.2330482672899115,
  -1.2009726393042142,
  -1.2087384562530576,
  0.1,
  0.1,
  0.02],
 [-0.31753842656452286,
  0.3776525353380408,
 